# 引言

Python函数提供了许多能够简化编程工作的特性，有些是Pythton特有的。这些特性能够更明确地体现出函数的目标。

# #19 不要把函数返回的多个值拆分到三个以上的变量中

拆包机制允许Python函数返回一个以上的值。假如现在要统计一群鳄鱼的各项指标，把每条鳄鱼的体长都保存在列表里。接着要编写函数，查出列表中最短与最长的鳄鱼。可以用下面这种写法实现，它可以同时把这两个值都返回。

In [1]:
def get_stats(numbers):
    minimum = min(numbers)
    maximum = max(numbers)
    return minimum, maximum

lengths = [63, 73, 72, 60, 67, 66, 71, 61, 72, 70]

minimum, maximum = get_stats(lengths)  # Two return values

print(f'Min: {minimum}, Max: {maximum}')

Min: 60, Max: 73


函数返回的其实是个元组。相当于用这两个变量分别接收元组中的两个元素。下面演示一下拆包语句和返回多个值的函数是怎么使用的。

In [2]:
first, second = 1, 2
assert first == 1
assert second == 2

def my_function():
    return 1, 2

first, second = my_function()
assert first == 1
assert second == 2

==在返回多个值的时候，可以用带星号的表达式接收那些没有被普通变量捕获到的值。==
例如，我们还要写一个函数，计算每条鳄鱼的长度与这些鳄鱼的平均长度之比。该函数会把比值放到列表里返回，我们可以只接受最长与最短的鳄鱼所对应的比值，其他的用带星号的写法总括。

In [3]:
def get_avg_ratio(numbers):
    average = sum(numbers) / len(numbers)
    scaled = [x / average for x in numbers]
    scaled.sort(reverse=True)
    return scaled

longest, *middle, shortest = get_avg_ratio(lengths)

print(f'Longest:  {longest:>4.0%}')
print(f'Shortest: {shortest:>4.0%}')

Longest:  108%
Shortest:  89%


假设现在需求又变了，我们这次还想知道平均长度、中位长度以及样本的总数。我们可以扩展原来的`get_stats`函数，然后用元组返回。

In [4]:
def get_stats(numbers):
    minimum = min(numbers)
    maximum = max(numbers)
    count = len(numbers)
    average = sum(numbers) / count

    sorted_numbers = sorted(numbers)
    middle = count // 2
    if count % 2 == 0:
        lower = sorted_numbers[middle - 1]
        upper = sorted_numbers[middle]
        median = (lower + upper) / 2
    else:
        median = sorted_numbers[middle]

    return minimum, maximum, average, median, count

minimum, maximum, average, median, count = get_stats(lengths)

print(f'Min: {minimum}, Max: {maximum}')
print(f'Average: {average}, Median: {median}, Count {count}')

assert minimum == 60
assert maximum == 73
assert average == 67.5
assert median == 68.5
assert count == 10

Min: 60, Max: 73
Average: 67.5, Median: 68.5, Count 10


这样写有两个问题。首先，函数返回的五个值都是数字，所以很容易搞错顺序。调用方同时接收这么多返回值，也容易出错。
第二问题是，调用函数并拆分返回值的那行代码会非常长，按照PEP8风格指南，可能需要折行，这让代码看起来很丑陋。

In [5]:
minimum, maximum, average, median, count = get_stats(
    lengths)

minimum, maximum, average, median, count = \
    get_stats(lengths)

(minimum, maximum, average,
 median, count) = get_stats(lengths)

(minimum, maximum, average, median, count
    ) = get_stats(lengths)

为了避免这些问题，我们不应该把函数返回的多个值拆分到三个以上的变量里。一个三元组最多只能拆成三个普通变量，或两个普通变量与一个万能变量(带星号的变量)。

如果要拆分的值比较多，那么还是定义一个命名元组或轻便的类。

# #20 遇到意外状况时应该抛出异常，不要返回None

编写工具函数时，许多人都喜欢用`None`这个返回值表示特殊情况。这或许有几分道理，例如，编写一个辅助函数计算两数相除的结果，在除数为0的情况下，返回`None`似乎很合理。

In [6]:
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

调用这个函数时，可以按照自己的方式处理这样的返回值。

In [7]:
x, y = 1, 0
result = careful_divide(x, y)
if result is None:
    print('Invalid inputs')

Invalid inputs


如果传给`careful_divide`的被除数为0，这时应该返回为0。问题是，在`if`条件判断中，可能会根据值本身是否相当于`False`来做判断：

In [8]:
x, y = 0, 5
result = careful_divide(x, y)
if not result:
    print('Invalid inputs') 

Invalid inputs


这种情况，会把返回0也当成返回`None`那样处理。因此`careful_divide`这样，用`None`来表示特殊状况的函数很容易出错。有两种办法可以减少这样的错误。

第一种办法是，利用二元组把计算结果分成两部分返回。元组的首个元素表示是否操作成功，第二个元素表示计算的实际值。
但是，有点麻烦。

第二种办法是，不用`None`表示特例，而是向调用方抛出异常。

In [9]:
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')

现在，调用方拿到函数的返回值之后，不用判断操作是否成功了。只要用`try`把函数包起来。

In [11]:
x, y = 5, 2
try:
    result = careful_divide(x, y)
except ValueError:
    print('Invalid inputs')
else:
    print('Result is %.1f' % result)

Result is 2.5


这个办法也可以扩展到那些使用类型注解的代码中，我们可以把函数的返回值指定为`float`类型，这样它就不可能返回`None`了。然而，Python采用的是动态类型与静态类型相搭配的gradual类型系统，我们不能再函数的接口上指定函数可能抛出哪些异常(像Java的受检异常)。所以，我们只好把有可能抛出的异常写在文档里面，并希望调用方能根据文档适当地捕获相关异常。

In [12]:
def careful_divide(a: float, b: float) -> float:
    """Divides a by b.
    Raises:
        ValueError: When the inputs cannot be divided.
    """
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')

这样写，输入、输出与异常都显得很清晰，所以调用方出错的概率就变得很小了。

# #21 了解如何在闭包里面使用外围作用域中的变量

有时，我们要给列表中的元素排序，而且要优先把某个群组之中的元素放在其他元素的前面。
实现这种做法的一种常见方案，是把辅助函数通过`key`参数传给列表的`sort`方法，让这个方法根据辅助函数所返回的值来决定元素在列表中的先后顺序。
辅助函数先判断当前元素是否处在重要的群组里，如果在，就把返回值的第一项写成0，让它能够排在不属于这个组的那些元素之前。

In [13]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)

该函数可以处理比较简单的输入数据。

In [14]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


它为什么能实现这个功能呢？要分三个原因来讲：
1. Python支持闭包(closure)，这让定义在大函数里面的小函数也能引用大函数之中的变量。此例中，`helper`函数能引用`group`参数。
2. 函数在Python里是头等对象(first-class object)，所以你可以像操作其他对象那样，直接引用它们、把它们赋值给变量、将它们当成参数传给其他函数，或是在`in`表达式与`if`语句里面对它做比较，等等。闭包函数也是函数，所以，同样可以传给`sort`方法的`key`参数。
3. Python在判断两个序列(包括元组)的大小时。它首先比较0号位置的那两个元素，如果相等，就比较1号位的那两个元素；以此类推，直到得出结论为止。所以，我们可以利用这套规则让`helper`这个闭包函数返回一个元组，并把关键指标写成元组的首个元素表示当前排序的值是否需要优先。

如果这个`sort_priority`函数还能告诉我们，列表里面有没有位于重要群组之中的元素，那就更好了，因为这样可以让调用者更方便地作出相应处理。
添加这样一个功能似乎相当简单，因为闭包函数本身就需要判断当前值是否处于重要群组之中，既然这样，那么不妨让它在发现这种值时，顺便把标志变量翻转过来。最后，让闭包外的大函数返回这个标志变量，如果闭包函数当时遇到过这样的值，那么这个标志肯定是`True`。

In [15]:
def sort_priority2(numbers, group):
    found = False
    def helper(x):
        if x in group:
            found = True  # Seems simple
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

我们还是用刚才的输入数据来运行这个新函数。

In [16]:
found = sort_priority2(numbers, group)
print('Found:', found)
print(numbers)

Found: False
[2, 3, 5, 7, 1, 4, 6, 8]


排序结果没有问题。但是表示函数返回值的`found`变量应该为`True`，我们却看到`False`，为什么？

在表达式中引用某个变量时，Python解释器会按照下面的顺序，在各个作用域里面查找这个变量，以解析这次引用。
1. 当前函数的作用域
2. 外围作用域(例如包含当前函数的其他函数所对应的作用域)。
3. 包含当前代码的那个模块所对应的作用域(也叫全局作用域)。
4. 内置作用域(build-in scope)，也就是包含`len`与`str`等函数的那个作用域。

如果这些作用域中都没有定义名称相符合的变量，那么程序就抛出`NameError`异常。

In [17]:
foo = dese_not_exist * 5

NameError: name 'dese_not_exist' is not defined

刚才讲的是遍历出现在赋值符号(=)右边时，该怎么认定。现在讲变量出现在赋值符号左边时(变量赋值)，该怎么处理。这要分两种情况处理，如果变量已经定义在当前作用域中，那么直接把新值交给它就行了。
如果当前作用域中不存在这个变量，那么即便外围作用域里有同名的变量，Python也还是会把这次的赋值操作当成变量的定义来处理，这会产生一个重要的效果，Python会把包含赋值操作的这个函数当成新定义的这个变量的作用域。

这解释了刚才那种写法错在何处。`sort_priority2`函数里面的`helper`闭包函数是把`True`赋值给了变量`found`。当前作用域里面没有这样一个叫`found`的变量，所以就算外围的`sort_priority2`函数里面有`found`变量，系统也还是会把这次赋值当成定义，也就是会在`helper`里面定义一个新的`found`变量，而不是把它当成给`sort_priority2`已有的那个`found`变量赋值。

In [18]:
def sort_priority2(numbers, group):
    found = False         # 作用域: 'sort_priority2'
    def helper(x):
        if x in group:
            found = True  # 作用域: 'helper' -- Bad!
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

这种问题有时也称为作用域bug(scoping bug)，Python新手可能会认为这样的赋值规则很奇怪，但实际上Python是故意这样设计的。
因为这样可以防止函数中的局部变量污染外围模块。假如不这样做，那么函数里的每条赋值语句都有可能影响全局作用域的变量，这样不仅混乱，而且会让全局变量之间彼此交互影响，从而导致很多难以探查的bug。